In [8]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from b_feature_engineering import df_prelag, df_clolag
import pandas as pd

In [9]:
# This function evaluate xgboost model for the initial train size and iterating forward creating models for each day
# Prediction is done for each model using model created in each iteration
def xgboost_model(df, train_size = 365, target = '', index = 'date'):
    preds = []
    actuals = []
    dates = []
    X = df.drop(columns= [index, target])
    Y = df[target]
    model = (XGBRegressor(
            objective="reg:squarederror",
            max_depth=7,
            learning_rate=0.001,
            n_estimators=150,
            random_state=42))
    
    for i in range(train_size, len(df)):
        X_train = X.iloc[:i]
        y_train = Y.iloc[:i]
        X_test = X.iloc[i:i+1]
        y_test = Y.iloc[i:i+1]

        # simple model: linear regression
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        preds.append(y_pred[0])
        actuals.append(y_test.values[0])
        dates.append(df.loc[i,index])

    df_results = (pd.DataFrame({
                    "date": dates,
                    "actual": actuals,
                    "predicted": preds }))
    return df_results

In [10]:
df = df_prelag.drop(columns=["closing_price", "clolag_1"]).copy()
df_eval = xgboost_model(df = df , target = 'price_diff')
df_eval.head(2)

,date,actual,predicted
0,2024-07-17,1.06,0.037638
1,2024-07-18,-0.84,0.081081


In [12]:
# ---- Evaluate performance ----
rmse = root_mean_squared_error(df_eval.actual, df_eval.predicted)
print(f"Walk-forward Root_Mean_Squared_Error (RMSE): {rmse:.2f}")

mae = mean_absolute_error(df_eval.actual, df_eval.predicted)
print(f"Walk-forward Mean Absolute Error (MAE): {mae:.2f}")

Walk-forward Root_Mean_Squared_Error (RMSE): 2.13
Walk-forward Mean Absolute Error (MAE): 1.59


XGBOOST model performed similar to the linear model but less accurate than first base line model.